<a href="https://colab.research.google.com/github/Masterjun12/Road_detection/blob/main/%EC%B0%A8%EC%84%A0%ED%9B%BC%EC%86%90_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- dataset/
    - train/
        - good/
            - good_image1.jpg
            - good_image2.jpg
            - ...
        - bad/
            - bad_image1.jpg
            - bad_image2.jpg
            - ...
    - filtered_good/ (분류된 적합한 이미지가 여기에 저장됨)
    - filtered_bad/ (분류된 부적합한 이미지가 여기에 저장됨)

    - to_classify/
    - image1.jpg
    - image2.jpg
    - ...

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# 데이터 디렉토리 경로 설정
data_dir = "/content/drive/MyDrive/dataset"  # 데이터셋 디렉토리 경로 설정
train_dir = os.path.join(data_dir, "train")  # 훈련 데이터 디렉토리
filtered_good_dir = os.path.join(data_dir, "/content/drive/MyDrive/dataset/filtered_good")  # 적합한 이미지 저장 디렉토리
filtered_bad_dir = os.path.join(data_dir, "/content/drive/MyDrive/dataset/filtered_bad")  # 부적합한 이미지 저장 디렉토리

# 이미지 데이터 생성기 설정
batch_size = 32
image_size = (150, 150)

train_data_gen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # 이미지 값을 0~1 사이로 조정
    validation_split=0.2  # 검증 데이터 분할 비율
)

train_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",  # 2개 클래스: 적합, 부적합
    subset="training"  # 훈련 데이터 생성기
)

validation_generator = train_data_gen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="binary",
    subset="validation"
)

# 모델 생성 및 훈련
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

epochs = 10
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

import shutil

# 분류할 이미지가 있는 디렉토리
to_classify_dir = "/content/drive/MyDrive/dataset/to_classify"

# 분류된 이미지 저장
for filename in os.listdir(to_classify_dir):
    if filename.endswith(".jpg"):
        img_path = os.path.join(to_classify_dir, filename)
        img = keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # 배치 차원 추가

        predictions = model.predict(img_array)
        if predictions[0] > 0.5:
            # 적합한 이미지
            destination_dir = filtered_good_dir
        else:
            # 부적합한 이미지
            destination_dir = filtered_bad_dir

        destination_path = os.path.join(destination_dir, filename)
        os.rename(img_path, destination_path)

# 모델 저장
model.save("/content/drive/MyDrive/dataset/image_classifier_model.h5")


Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Epoch 1/10
5/5 [==============================] - 48s 10s/step - loss: 7.3229 - accuracy: 0.5125 - val_loss: 2.8282 - val_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 8s 1s/step - loss: 3.4536 - accuracy: 0.5938 - val_loss: 3.6568 - val_accuracy: 0.5000
Epoch 3/10
5/5 [==============================] - 9s 2s/step - loss: 2.5416 - accuracy: 0.5750 - val_loss: 0.7239 - val_accuracy: 0.7000
Epoch 4/10
5/5 [==============================] - 7s 1s/step - loss: 1.3683 - accuracy: 0.5625 - val_loss: 0.7532 - val_accuracy: 0.6250
Epoch 5/10
5/5 [==============================] - 8s 1s/step - loss: 0.4686 - accuracy: 0.8062 - val_loss: 1.0031 - val_accuracy: 0.6250
Epoch 6/10
5/5 [==============================] - 9s 2s/step - loss: 0.4454 - accuracy: 0.7875 - val_loss: 0.4539 - val_accuracy: 0.8000
Epoch 7/10
5/5 [==============================] - 7s 1s/step - loss: 0.1572 - accuracy: 0.9500 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [4]:
!pip install tensorflow==2.14.0rc0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 89.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
